In [2]:
from generic_qaoa.vqf_helper import factor_56153
from generic_qaoa.clause import MathematicalClause
from generic_qaoa.utils import get_pq_from_selected, plot_histogram
from generic_qaoa import GenericQaoa
%matplotlib inline


In [3]:
m = 56153
p_dict, q_dict, z_dict, clauses = factor_56153()
free_symbols = set().union(*[clause.free_symbols for clause in clauses])
qubit_index_to_symbol = {i: sym for i, sym in enumerate(free_symbols)}
symbol_to_qubit_index = {sym: i for i, sym in qubit_index_to_symbol.items()}
final_clauses = [MathematicalClause((clause * clause).expand(), symbol_to_qubit_index)for clause in clauses if clause != 0]

In [4]:
vqf = GenericQaoa(_p=3,
                  _clauses=final_clauses,
                  _qbits=range(len(free_symbols)),
                  _grid_size=8,
                  simulate=True)

vqf.run()
plot_histogram(vqf.last_result.counts_histogram)
p, p_dict, q, q_dict = get_pq_from_selected(p_dict, q_dict, vqf.last_result.selected, symbol_to_qubit_index)
print("p,q=", p, q)

/Users/snir/HW/generic_qaoa/.venv/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/snir/HW/generic_qaoa/.venv/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circuit on aer_simulator
done
running circ

In [5]:
if p * q != m:
    print("Trying to fix with bit-flip.")
    for i in range(len(p_dict)):
        for j in range(len(q_dict)):
            new_p: int
            if p_dict[i] == 1:
                new_p = p - 2 ** i
            else:
                new_p = p + 2 ** i
            new_q: int
            if q_dict[j] == 1:
                new_q = q - 2 ** j
            else:
                new_q = q + 2 ** j
            if new_q == m or new_p == m:
                break
            if new_p * q == m:
                p = new_p
            elif p * new_q == m:
                q = new_q
            if new_p * new_q == m:
                p = new_p
                q = new_q
            if p * q == m:
                break
        if p * q == m:
            break
print(p,q)

233 241
